In [1]:
import os, csv
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [3]:
### line alert function
def notifyMsg(*msg, url=LINE_URL, headers=LINE_HEADERS):
    r = requests.post(url, headers=headers, data={'message': msg[0]})
    print(r.text)

In [4]:
### create technical strategy
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 10},
        {'kind': 'atr', 'length': 15}
    ]
)

In [5]:
def initial_bars(pair='EURUSD', tf='15min'):
    bars, _ = ts.get_intraday(symbol=pair,interval=tf, outputsize='full')
    df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with strategy
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.iloc[::-1] # reverse row
    df.reset_index(drop=True, inplace=True)
        
    order = {}
    current_action = 'close'
    
    return df , order, current_action

In [74]:
def get_realtime(df=None, order={}, current_action={}, pair='EURUSD', tf='15min', ema_len=200, rsi_len=14, atr_len=14, \
                 strategy=None, overbought=70, oversold=30, rr=1.5, pip_value=0.0001):
    
    bars, _ = ts.get_intraday(symbol=pair,interval=tf, outputsize='compact')
    incoming_df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with rsi strategy
    
    incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
    if (incoming_df['timestamp'].iloc[0] != df['timestamp'].iloc[-1]):
        df = df[:900].append(incoming_df.head(1), ignore_index = True)
        
#     df = df.iloc[::-1] # reverse row
#     df.reset_index(drop=True, inplace=True)
#     if strategy != None: df.ta.strategy(strategy)
    rsi_o2 = ta.Strategy(
        name = 'RSI 14 Days for opposite trade',
        description = 'RSI for alert',
        ta = [
            {'kind': 'ema', 'length': ema_len},
            {'kind': 'rsi', 'length': rsi_len},
            {'kind': 'atr', 'length': atr_len}
        ]
    )
    df.ta.strategy(rsi_o2)
    
#     orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L','result'])
#     current_action = 'close'
#     order = {}
    
    row = df.tail(1)
    return row
    if current_action[pair] == 'close' or order[pair] == {}:
        if (row['EMA_'+str(ema_len)] < row['low']).bool() and (row['RSI_'+str(rsi_len)] > overbought).bool():
            row['action'] = 'buy'
            order[pair]['action'] = 'buy'
            order[pair]['open time'] = row['timestamp']
            order[pair]['open'] = row['open']
            order[pair]['S/L'] = row['low'] - row['ATRr_'+str(atr_len)]
            order[pair]['T/P'] = row['open'] + (rr * abs(row['open'] - order[pair]['S/L']))
            current_action[pair] = 'buy'
            notifyMsg(f"Buy {pair} RSI_O2\nprice: {str(order[pair]['open'])}\nTP: {str(order[pair]['T/P'])}\nSL: {str(order[pair]['S/L'])}")
            
        elif (row['EMA_'+str(ema_len)] > row['high']).bool() and (row['RSI_'+str(rsi_len)] < oversold).bool():   
            row['action'] = 'sell'
            order[pair]['action'] = 'sell'
            order[pair]['open time'] = row['timestamp']
            order[pair]['open'] = row['open']
            order[pair]['S/L'] = row['high'] + row['ATRr_'+str(atr_len)]
            order[pair]['T/P'] = row['open'] - (rr * abs(row['open'] - order[pair]['S/L']))
            current_action[pair] = 'sell'
            notifyMsg(f"Sell {pair} RSI_O2\nprice: {str(order[pair]['open'])}\nTP: {str(order[pair]['T/P'])}\nSL: {str(order[pair]['S/L'])}")
        
    if current_action[pair] == 'buy':
        if row['low'] <= order[pair]['S/L']:
            order[pair]['result'] = 'S/L'
            order[pair]['close time'] = row['timestamp']
            order[pair]['close'] = order[pair]['S/L']
            current_action[pair] = 'close'
            order[pair] = {}
            notifyMsg(f"S/L Buy {pair} RSI_O2\nprice: {order[pair]['close']}")
        elif row['high'] >= order[pair]['T/P']:
            order[pair]['result'] = 'T/P'
            order[pair]['close time'] = row['timestamp']
            order[pair]['close'] = order['T/P']
            current_action[pair] = 'close'
            order[pair] = {}
            notifyMsg(f"T/P Buy {pair} RSI_O2\nprice: {order[pair]['close']}")
                
    elif current_action[pair] == 'sell':
        if row['high'] >= order[pair]['S/L']:
            order[pair]['result'] = 'S/L'
            order[pair]['close time'] = row['timestamp']
            order[pair]['close'] = order[pair]['S/L']
            current_action[pair] = 'close'
            order[pair] = {}
            notifyMsg(f"S/L Sell {pair} RSI_O2\nprice: {order[pair]['close']}")
        elif row['low'] <= order[pair]['T/P']:
            order[pair]['result'] = 'T/P'
            order[pair]['close time'] = row['timestamp']
            order[pair]['close'] = order[pair]['T/P']
            current_action[pair] = 'close'
            order[pair] = {}
            notifyMsg(f"T/P Sell {pair} RSI_O2\nprice: {order[pair]['close']}")
            
    
# r = get_realtime()

In [75]:
pair_action = {}
pair_order = {}
eurusd, pair_order['EURUSD'], pair_action['EURUSD'] = initial_bars(pair='EURUSD')
gbpusd, pair_order['GBPUSD'], pair_action['GBPUSD'] = initial_bars(pair='GBPUSD')
usdjpy, pair_order['USDJPY'], pair_action['USDJPY'] = initial_bars(pair='USDJPY')

In [79]:
def get_all_realtime():
    get_realtime(df=gbpusd, pair='GBPUSD', order=pair_order, current_action=pair_action, \
                 rsi_len=20, atr_len=10, overbought=70, oversold=30) # rr 1:1   10 pip=0.001
    
    get_realtime(df=usdjpy, pair='USDJPY', order=pair_order, current_action=pair_action, \
                 rsi_len=12, atr_len=17, overbought=73, oversold=27) # rr 1:1.1 10 pip=0.1
    
    get_realtime(df=eurusd, pair='EURUSD', order=pair_order, current_action=pair_action, \
                 rsi_len=10, atr_len=15, overbought=76, oversold=24) # rr 1:1   10 pip=0.001
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-02-03 19:42:38)

In [80]:
while True:
    schedule.run_pending()
    time.sleep(1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### === test code below ===

In [6]:
data, _ = cc.get_currency_exchange_rate(from_currency='EUR',to_currency='USD')
ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [7]:
data

{'1. From_Currency Code': 'EUR',
 '2. From_Currency Name': 'Euro',
 '3. To_Currency Code': 'USD',
 '4. To_Currency Name': 'United States Dollar',
 '5. Exchange Rate': '1.12397430',
 '6. Last Refreshed': '2022-02-01 06:43:01',
 '7. Time Zone': 'UTC',
 '8. Bid Price': '1.12397430',
 '9. Ask Price': '1.12397430'}

In [8]:
data_ts,_ = ts.get_intraday(symbol='EURUSD',interval='30min', outputsize='compact')
data_ts

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-02-01 02:00:00,1.1238,1.1239,1.1234,1.1235,0.0
2022-02-01 01:30:00,1.1240,1.1243,1.1233,1.1238,0.0
2022-02-01 01:00:00,1.1243,1.1245,1.1237,1.1240,0.0
2022-02-01 00:30:00,1.1243,1.1245,1.1239,1.1243,0.0
2022-02-01 00:00:00,1.1238,1.1245,1.1235,1.1243,0.0
...,...,...,...,...,...
2022-01-28 02:00:00,1.1150,1.1155,1.1145,1.1147,0.0
2022-01-28 01:30:00,1.1154,1.1155,1.1148,1.1149,0.0
2022-01-28 01:00:00,1.1153,1.1155,1.1148,1.1154,0.0


In [49]:
t_order = {}
p = 'eurusd'
t_order['eurusd'] = {'entry': 10, 'action': 'buy'}
t_order['gbpusd'] = {'entry': 20, 'action': 'buy'}
t_order[p]['entry']

10

In [ ]:
all_jobs = schedule.get_jobs()
all_jobs